# **Fine-tuning GPT2 for text generation**
### This notebook has been taken from [here](https://gist.github.com/GeorgeDittmar/5c57a35332b2b5818e51618af7953351)
- It contains code to fine tune GPT-2 on two datasets:
  1. [Tiny Shakespeare](https://huggingface.co/datasets/tiny_shakespeare) - 40,000 lines of Shakespeare from a variety of Shakespeare's plays
  2. [Bill Sum](https://huggingface.co/datasets/billsum) - Summarization of US Congressional and California state bills.


## Table of contents
1. [Imports and Installation section](#Imports)
2. [Shakespeare Data preparation](#Shakespeare)
3. [GPT-2 Model Fine tuning on Shakespeare Dataset](#FineTuning)
4. [Shakespeare Text Generation using GPT-2](#TextGeneration) 
5. [Bill Sum Data set preparation](#BillSum)
6. [GPT-2 Model Fine tuning on Bill Sum Dataset](#FineTuningBillSum)
7. [Bill Sum Text Generation using GPT-2](#BillSumTextGeneration) 

## Part 1: Imports and Installation section <a name="Imports"></a>

In [ ]:
# import necessary modules
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 62213 (delta 3), reused 9 (delta 0), pack-reused 62194
Receiving objects: 100% (62213/62213), 47.44 MiB | 28.97 MiB/s, done.
Resolving deltas: 100% (44035/44035), done.


In [ ]:
# Clone should now be in the machine
!ls

drive  sample_data  transformers


Change directory location to be in the examples folder and then install any requirements

In [ ]:
os.chdir("transformers")
os.chdir("./examples/language-modeling")
!ls

README.md	  run_clm.py	   run_mlm.py
requirements.txt  run_mlm_flax.py  run_plm.py


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 163kB 17.1MB/s 
     |████████████████████████████████| 1.2MB 47.0MB/s 
     |████████████████████████████████| 20.7MB 1.2MB/s 
     |████████████████████████████████| 245kB 61.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!ls

README.md	  run_clm.py	   run_mlm.py
requirements.txt  run_mlm_flax.py  run_plm.py


In [ ]:
!pip install pyarrow --upgrade

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (3.0.0)


In [ ]:
os.chdir("/content/transformers/examples/")
os.chdir("./language-modeling")

In [ ]:
# Need to install latest transformer packages from github so the scripts will run correctly
!pip install git+git://github.com/huggingface/transformers/

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-txc9i948
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-txc9i948
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 16.9MB/s 
     |████████████████████████████████| 890kB 59.3MB/s 
  Created wheel for transformers: filename=transformers-4.4.0.dev0-cp36-none-any.whl size=1808787 sha256=1977580b98abbeaf11ff53ffbeb24ea299c7950511eca6740b420fb5ea1a4e2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4p5t8ob0/wheels/dc/e5/1e/3a2977a646558fca07585cadcf56aa4a910e995ba945961c4e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2b8048d1b925501e3ea9b7ac81da1ab91eef176a61fbad92c3116bdb59c97c4d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e

## Part 2: Shakespeare Data Preparation section<a name="Shakespeare"></a>
- Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [ ]:
with open('/content/drive/MyDrive/bill_sum_shakespeare/shakespeare.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size = 0.9, random_state = 42)
print("Training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

Training size:36000
Evaluation size: 4000


## Part 3: GPT-2 Model Fine tuning on Shakespeare Dataset<a name="FineTuning"></a>
### Fine tuning with **1 epoch**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 1 \
--fp16 \
--learning_rate 5e-5 \
--output_dir="Output dir"

2021-02-10 14:01:11.237413: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/10/2021 14:01:12 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/10/2021 14:01:12 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Output dir>, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Feb10_14-01-12_cf74782d0fe2, logging_first_step=False, logging_steps=500, save_steps=-1, save_total_limit=None

### Fine tuning with **3 epochs**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 3 \
--fp16 \
--learning_rate 5e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-09 10:27:58.071755: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/09/2021 10:27:59 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/09/2021 10:27:59 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb09_10-27-59_aa1c1fd0c52e, logging_first_step=False, logging_steps=500, save_steps=-1, save

### Fine tuning with **5 epochs**


In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--learning_rate 5e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-09 10:48:48.805022: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/09/2021 10:48:50 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/09/2021 10:48:50 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb09_10-48-50_aa1c1fd0c52e, logging_first_step=False, logging_steps=500, save_steps=-1, save

### Fine tuning with **7 epochs**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 7 \
--fp16 \
--learning_rate 5e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-09 11:20:44.331601: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/09/2021 11:20:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/09/2021 11:20:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=7.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb09_11-20-45_aa1c1fd0c52e, logging_first_step=False, logging_steps=500, save_steps=-1, save

### Fine tuning with **Learning rate = 4e-5** & number of **epochs = 5**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--learning_rate 4e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-09 12:02:47.056848: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/09/2021 12:02:48 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/09/2021 12:02:48 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=4e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb09_12-02-48_aa1c1fd0c52e, logging_first_step=False, logging_steps=500, save_steps=-1, save

#### Fine tuning with **Learning rate = 3e-5** & number of **epochs = 5**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--learning_rate 3e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-09 12:31:54.267605: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/09/2021 12:31:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/09/2021 12:31:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb09_12-31-56_aa1c1fd0c52e, logging_first_step=False, logging_steps=500, save_steps=-1, save

#### Fine tuning with **Learning rate = 2e-5** & number of **epochs = 5**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--learning_rate 2e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-09 13:01:03.892428: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/09/2021 13:01:05 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/09/2021 13:01:05 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb09_13-01-05_aa1c1fd0c52e, logging_first_step=False, logging_steps=500, save_steps=-1, save

In [ ]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("Output dir", from_pt = True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.22.attn.masked_bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.20.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.17.attn.masked_bias', 'lm_head.weight', 'transformer.h.14.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.21.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.23.attn.masked_bias']
-

## Part 4: Shakespeare Text Generation using GPT-2 <a name="TextGeneration"></a>

In [ ]:
# set the initial text to start the process of text generation
input_ids = tokenizer.encode("He that will give good words to thee will", return_tensors = 'tf')

# print the tensor ids
input_ids[0]

<tf.Tensor: shape=(9,), dtype=int32, numpy=
array([ 1544,   326,   481,  1577,   922,  2456,   284, 17903,   481],
      dtype=int32)>

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length = 128,  
    num_return_sequences = 5,
    no_repeat_ngram_size = 2,
    repetition_penalty = 1.5,
    top_p = 0.92,
    temperature = 0.85,
    do_sample = True,
    top_k = 125,
    early_stopping = True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [ ]:
# Print output for each sequence generated above
for i, token in enumerate(generated_text_samples):
  print("{}: {}".format(i, tokenizer.decode(token, skip_special_tokens = True)))
  print()

0: He that will give good words to thee will also deliver thy spirit into the hands of God. - Psalm 83:20.
There is a reason for this prophecy, however; it points to Christ's return and His coming again on Earth upon his own behalf (see Hebrews 13). However how does he accomplish these goals by making himself available at once? And then why did Jesus bring back all those things which have been stolen from Him as He had already done so in Heaven before leaving us here today just years ago? On one side are various Christian saints who would not tolerate their former status with our modern world but still remain faithful

1: He that will give good words to thee will be more profitable for me than any king. And let him who has been made a great man say unto his disciples, O men! I have laid before you all the kingdom of Heaven; and thy place is this: ye shall not go down from it until thou canst see clearly what my wrath hath caused iniquity by those things which are done among us."
17 The

## Part 5: Bill Sum Data set preparation<a name="BillSum"></a>

In [ ]:
# read the training data into pandas dataframe
data_path = '/content/drive/MyDrive/bill_sum_shakespeare/us_train_data_final_OFFICIAL.jsonl'
data = pd.read_json(data_path, lines = True)

# display settings
pd.set_option('display.max_colwidth', None)

# display few rows of the dataframe
data.head(2)

bill_id  ... sum_len
0  107_hr2256  ...     527
1  111_hr4710  ...    1161

[2 rows x 6 columns]

In [ ]:
# check the shape of the data frame
data.shape

(18949, 6)

In [ ]:
# extract the text data
text = data['summary'].values.tolist()

# split data into train and validation set
train, eval = train_test_split(text, train_size = 0.9, random_state = 42)
print("Training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

Training size:17054
Evaluation size: 1895


## Part 6: GPT-2 Model Fine tuning on Bill Sum Dataset<a name="FineTuningBillSum"></a>
### Fine tuning with **1 epoch**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 1 \
--fp16 \
--learning_rate 5e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-10 01:23:20.838664: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/10/2021 01:23:22 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/10/2021 01:23:22 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb10_01-23-22_db3590ecbfe2, logging_first_step=False, logging_steps=500, save_steps=-1, save

### Fine tuning with **3 epochs**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 3 \
--fp16 \
--learning_rate 5e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-10 02:10:09.675636: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/10/2021 02:10:11 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/10/2021 02:10:11 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb10_02-10-11_db3590ecbfe2, logging_first_step=False, logging_steps=500, save_steps=-1, save

### Fine tuning with **5 epochs**

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--learning_rate 5e-5 \
--lr_scheduler_type constant \
--evaluation_strategy epoch \
--output_dir="Output dir" \
--overwrite_output_dir True

2021-02-10 04:21:35.962423: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/10/2021 04:21:37 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/10/2021 04:21:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0, logging_dir=runs/Feb10_04-21-37_db3590ecbfe2, logging_first_step=False, logging_steps=500, save_steps=-1, save

## Part 7: Bill Sum Text Generation using GPT-2 <a name="BillSumTextGeneration"></a>


In [ ]:
model = TFGPT2LMHeadModel.from_pretrained("Output dir", from_pt = True)

# set the initial text to start the process of text generation
bill_sum_input_ids = tokenizer.encode("This Act may be cited as the ``Farm to School Improvements Act", return_tensors = 'tf')

# print the tensor ids
bill_sum_input_ids[0]

<tf.Tensor: shape=(13,), dtype=int32, numpy=
array([ 1212,  2191,   743,   307,  9181,   355,   262,  7559, 48412,
         284,  3961, 45097,  2191], dtype=int32)>

### Generate Text

In [ ]:
bill_sum_generated_text_samples = model.generate(
    bill_sum_input_ids, 
    max_length = 128,  
    num_return_sequences = 5,
    no_repeat_ngram_size = 2,
    repetition_penalty = 1.5,
    top_p = 0.92,
    temperature = 0.85,
    do_sample = True,
    top_k = 125,
    early_stopping = True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [ ]:
# Print output for each sequence generated above
for i, token in enumerate(bill_sum_generated_text_samples):
  print("{}: {}".format(i, tokenizer.decode(token, skip_special_tokens = True)))
  print()

0: This Act may be cited as the ``Farm to School Improvements Act''. [[Page 127 STAT. 1294]] (2) <<NOTE: Deadline.>> Effective date.--The amendment made by paragraph 18 of section 1346(e)(4), and in addition, all amendments making such an award under this subtitle shall take effect on September 1st 2009. SEC., 2013A--FREEDOM OF HAWTHORNE COUNTY REGISTER AND RIGHTS ASSESSMENTS WITH RESPECT TO THE PURPOSE FOR SHALLOWING A GRASS FIBERATION PROGRAMS OR ADMISSION REQUIREMENT BY RESIDENTS COMPET

1: This Act may be cited as the ``Farm to School Improvements Act''. SEC. 576A--MULTIPLE DISTRIBUTION ADMINISTRATION OF SMALL, UNIFORMED AND PROFIT-DEDUCTIBLE FINANCIAL SERVICES ACT of 2015 INTELLIGENCE AUTHORITY FOR FARMING LOCATIONS WITH LAND PROGRAMS THAT DO NOT PROVIDE SUBMARINE RESEARCH FUNDS TO OTHER SCHOOL REPORTS OR CONTRACTORS AS DEFINITIONALLY NEEDY IS COMMONLY DESIGNATED BY THE PRESIDENCY GOVERNMENT ARGUMENT AFFAIR D

2: This Act may be cited as the ``Farm to School Improvements Act''. (b